## <i class="fa fa-repeat"></i> 切换pip软件源

In [ ]:
import os
import subprocess

def check_pip_list():
    output = !pip config list
    if "global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'" in output and "global.trusted-host='pypi.tuna.tsinghua.edu.cn'" in output:
        return True
    else:
        return False
if not check_pip_list():
    # 查找pip.conf文件
    pip_conf_file = os.popen("find / -name 'pip.conf'").read().strip()
    # 直接覆盖pip.conf文件
    with open(pip_conf_file, 'w') as f:
        f.write("[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\ntrusted-host = pypi.tuna.tsinghua.edu.cn")
    # 验证修改是否成功
    if check_pip_list():
        print("pip软件源切换成功！")
    else:
        print("pip软件源切换失败，请检查脚本是否正确运行。")
else:
    print("pip软件源本来就是清华源，无需切换")

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    "iputils-ping",
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool
    


## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-arrow-down"></i> 下载并安装InvokeAI
手动安装方法出处：https://invoke-ai.github.io/InvokeAI/installation/020_INSTALL_MANUAL/#conda-method

In [ ]:
base_args = [
    proxy,'&&',
    'source activate','&&',
    'conda activate invokeai','&&',
    f'cd {InvokeAI_dir}','&&'
]

import os

InvokeAI_dir=os.path.join(root_dir,'InvokeAI')
InvokeAI_runtime_dir = os.path.join(root_dir,'InvokeAI_runtime')

if not os.path.exists(InvokeAI_dir):
    !$proxy &&\
    cd $root_dir &&\
    git clone https://github.com/invoke-ai/InvokeAI.git
else:
    print(f'{InvokeAI_dir}已经存在，无需克隆')

!cd $InvokeAI_dir &&\
ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml

env_yml_path=os.path.join(InvokeAI_dir,'environment.yml')
linux_env_yml_path=os.path.join(InvokeAI_dir,'environments-and-requirements/environment-lin-cuda.yml')

def check_env_installed():
    import subprocess
    # Use the subprocess module to run the "conda env list" command
    result = subprocess.run(["conda", "env", "list"], stdout=subprocess.PIPE)
    # Get the output of the command as a string and split it into a list of lines
    output = result.stdout.decode("utf-8")
    # Check if the "invokeai" environment is in the list
    print(output)
    return output

def configure_invokeai():
    # conda激活invokeai环境，且运行配置.py文件
    # 2023/1/16更新：遇到报错 AttributeError: module 'dns.rdtypes' has no attribute 'ANY' ，看解决方案：https://github.com/invoke-ai/InvokeAI/issues/2323
    configure_invokeai_args = base_args[:]
    configure_invokeai_args += [
        'pip install dnspython==2.2.1',
        '&&',
        f'cd {InvokeAI_dir}'
        '&&',
        f'python scripts/configure_invokeai.py --root_dir {InvokeAI_runtime_dir}'
    ]
    # 是否要进行默认全部yes的安装，The program was started with a '--yes' flag, which indicates user's acceptance of the above License terms
    # configure_invokeai_args += ['-y']
    
    configure_invokeai_cmd = (" ").join(configure_invokeai_args)
    print(configure_invokeai_cmd)
    copy_to_clipboard(configure_invokeai_cmd)
    print('目前不能完全自动化从头到尾安装，所以请在终端粘贴以上这段代码，然后在终端中运行')
    # 目前不能完全自动化从头到尾安装，所以请在终端粘贴以上这段代码，然后在终端中运行
    # !$configure_invokeai_cmd

if os.path.realpath(env_yml_path)==linux_env_yml_path:
    print(f'environment.yml已链接到 InvokeAI 根目录中，并且它指向 {linux_env_yml_path}')

    if "invokeai" not in check_env_installed():
        # If it is, activate the environment
        # 通过conda构建适合的python环境
        !$proxy &&\
        cd $InvokeAI_dir &&\
        conda env update
        
        if "invokeai" in check_env_installed():
            print('已经存在名为invokeai的虚拟环境')
            configure_invokeai()
    else:
        print('已经存在名为invokeai的虚拟环境')
        configure_invokeai()


## <i class="fa fa-wrench"></i> 安装 PyPatchMatch
pypatchmatch 是一个用于修复图像的 Python 模块。不需要运行InvokeAI，但它大大提高了修复和修复的质量，推荐使用。
不幸的是，它是一个 C++ 优化模块，安装可能有些困难。本指南将引导您完成这些步骤。
原文地址:https://invoke-ai.github.io/InvokeAI/installation/060_INSTALL_PATCHMATCH/#macintosh

### 安装必要工具

In [ ]:
def check_patchmatch_installed():
    install_pypatch_args = base_args[:] + [
        'python -c "from patchmatch import patch_match as pm;print(pm.patchmatch_available)"'
    ]
    install_pypatch_cmd = (" ").join(install_pypatch_args)
    pypatch_cb = !$install_pypatch_cmd
    if pypatch_cb[0] != 'True':
        print('pypatchmatch还没安装，即将安装')
        return False
    else:
        print('pypatchmatch已经安装，跳过安装')
        return True

patchmatch_installed = False
patchmatch_installed = check_patchmatch_installed()
print(patchmatch_installed)

if not patchmatch_installed:
    import subprocess

    libraries_to_install = [
    ]

    tools_to_install = [
        'build-essential',
        'python3-opencv',
        'libopencv-dev'
    ]

    installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

    # 尝试安装所有未安装的库
    for library in libraries_to_install:
        if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
            print(f"【 {library} 】已经安装，跳过安装")
            continue
        elif not any(library in installed_library for installed_library in installed_libraries):
            !pip install $library
        else:
            print(f"【 {library} 】已经安装，跳过安装")

    # 尝试安装所有未安装的工具
    update_needed = False
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            update_needed = True
            break
        else:
            print(f"【 {tool} 】已经安装，跳过安装")
    if update_needed:
        !apt-get update
        for tool in tools_to_install:
            exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
            if exit_code != 0:
                !apt-get install -y $tool

### 实装pypatchmatch并导入该模块进行检验

In [ ]:
if not patchmatch_installed:
    
    import os

    pkgconfig_dir='/usr/lib/x86_64-linux-gnu/pkgconfig/'
    if os.path.exists(pkgconfig_dir):
        !cd $pkgconfig_dir &&\
        ln -sf opencv4.pc opencv.pc

    #检查 opencv.pc 是否指向 opencv4.pc
    opencv_pc_path = os.path.join(pkgconfig_dir,'opencv.pc')
    opencv_pc4_path = os.path.join(pkgconfig_dir,'opencv4.pc')

    if os.path.realpath(opencv_pc_path)==opencv_pc4_path:
        print('opencv包配置文件的命名已修复')

    # 激活 invokeai 的环境并安装 pypatch
    install_pypatch_args = base_args[:] + [
        'pip install "git+https://github.com/invoke-ai/PyPatchMatch@0.1.3#egg=pypatchmatch"'
    ]

    install_pypatch_cmd = (" ").join(install_pypatch_args)

    !$install_pypatch_cmd

    check_installed_args = base_args[:] + [
        'python -c "from patchmatch import patch_match"'
    ]

    check_installed_cmd = (" ").join(check_installed_args)

    !$check_installed_cmd

## <i class="fa fa-hammer"></i> 配置invokeai.init文件
通过修改这里的参数来自定义invokeai的配置，如修改--outdir来改变invokeai生成的目标路径，修改--port来修改运行端口

In [ ]:
port = 6006

if env_name == 'AutoDL':
    invoke_outputs_dir='/root/autodl-nas/invoke_outputs'
    invokeai_init_path=os.path.join(InvokeAI_runtime_dir,'./invokeai.init')
elif env_name == 'OpenBayes':
    invoke_outputs_dir=os.path.realpath(os.path.join(root_dir,'./invoke_outputs'))
    invokeai_init_path=os.path.realpath(os.path.join(root_dir,'./invokeai/invokeai.init'))
    
import re

def add_line(lines, line_to_add):
    # 如果对应的参数和文本不存在，在文件末尾增加一行
    if not any(line.strip() == f'{line_to_add}' for line in lines):
        lines.append(f'{line_to_add}\n')

# 读取文件
with open(invokeai_init_path, 'r') as f:
    lines = f.readlines()

# 进行文本替换
for i, line in enumerate(lines):
    lines[i] = re.sub(r'--outdir="(.*)"', f'--outdir="{invoke_outputs_dir}"', line)
    lines[i] = re.sub(r'--nsfw_checker', f'--no-nsfw_checker', line)

#追加额外的参数
add_line(lines,'--no-nsfw_checker')
add_line(lines,'--no-safety_checker')
add_line(lines,f'--port={port}') #设定端口
add_line(lines,'--web') #以webui的形式启动

# 去除重复的行
seen = {}
result = []
for line in lines:
    if line in seen:
        continue
    seen[line] = True
    result.append(line)

lines = result
    
# 去除重复的行，但是这种做法会导致行序打乱
# lines = list(set(lines))

# 将修改后的内容写回文件
with open(invokeai_init_path, 'w') as f:
    f.writelines(lines)


## <i class="fa fa-eye"></i> 以WebUI的形式去启动InvokeAI

In [ ]:
final_args = []
final_args += base_args

git_pull_args = base_args[:] + ['git pull']
git_pull_cmd = (" ").join(git_pull_args)

git_pull_info = !$git_pull_cmd

print(git_pull_info)

if 'Already up to date.' not in git_pull_info:
    print('准备更新...')
    final_args.append([
        'conda env update','&&',
        f'python scripts/configure_invokeai.py -y --skip-sd-weights --root_dir {InvokeAI_runtime_dir}'
    ])
else:
    print('已经更新到最新，直接启动')
    
'''
在 Python 中，可以使用以下几种方法来拼接两个列表：
使用 + 操作符：new_list = list1 + list2。这会创建一个新的列表，其中包含原始两个列表中的所有元素。
使用内置函数 extend()：list1.extend(list2)。这会将 list2 中的所有元素添加到 list1 的末尾。
使用内置函数 sum()：new_list = sum([list1, list2], [])。这会将两个列表拼接起来，并将结果保存到新的列表中。
'''

final_args = base_args[:]
final_args += [
    f'python scripts/invoke.py --root_dir {InvokeAI_runtime_dir}',
]

final_cmd = (" ").join(final_args)
print(final_cmd)
!$final_cmd

## <i class="fa fa-arrow-down"></i>【即将废弃|无法安装成功】下载InvokeAI安装器

In [ ]:
# import requests
# import os
# import subprocess

# # Replace {OWNER} and {REPO} with the owner and repository name respectively
# url = f"https://api.github.com/repos/invoke-ai/InvokeAI/releases/latest"

# # Make the GET request to the GitHub API
# response = requests.get(url)

# # The response is a JSON object, which we can convert to a Python dictionary
# release_info = response.json()

# # print(release_info)
# # Access the desired information from the dictionary
# release_name = release_info["name"]
# tag_name = release_info["tag_name"]

# release_assets = release_info["assets"]

# invokeai_linux_installer_url=None

# for i in release_assets:
#     if 'linux' in i['browser_download_url']:
#         invokeai_linux_installer_url=i['browser_download_url']
        
# if invokeai_linux_installer_url:
#     file_name = os.path.basename(invokeai_linux_installer_url)
#     local_dir=os.path.join(root_dir,file_name)
#     installer_dir=os.path.join(root_dir,'InvokeAI-Installer')
#     install_sh_path=os.path.join(installer_dir,'install.sh')
    
#     if not os.path.exists(local_dir):
#         #下载安装器压缩包
#         !$proxy && wget -O $local_dir $invokeai_linux_installer_url
#     else:
#         print(f'本地已经存在 {file_name}，不进行下载')
        
#     if not os.path.exists(installer_dir):
#         #解压压缩包
#         !unzip -o $local_dir -d $root_dir
#     else:
#         print(f'本地已经存在 {installer_dir}，不进行解压')
        
#     if os.path.exists(install_sh_path):
#         print('正准备安装')

